In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv("C:/Users/surendar.reddy/Documents/data Science projet_Data traine/DataTrained_Project_Week-1/titanic_train.csv")
pd.set_option('display.max_rows', None)
df

In [ ]:
df.shape

In [ ]:
df.columns
#Survived-Label

In [ ]:
df.info()
#Need Encoding columns-sex, Embarked
#drop columns; Just for information-Name,ticket,Cabin(Not of Null Value)
#change to int65-Age

In [ ]:
df.isnull().sum()
#Age need Imputation-I prefer Deleting nan, because Age would be a very important parameter for influencing Survival rate

In [ ]:
#Encoding Done

from sklearn import preprocessing
LE=preprocessing.LabelEncoder()

df['Embarked']=LE.fit_transform(df['Embarked'])
df['Sex']=df['Sex'].replace({'male':1, 'female':0})


char_cabin=df['Cabin'].astype(str)
new_cabin=np.array([cabin[0]for cabin in char_cabin])
df['Cabin']=new_cabin

#As there are many nan and cannot delete them,  and practically knowing this would not influence the survival rate, Encoding this column
df['Cabin']=LE.fit_transform(df['Cabin'])


In [ ]:
df.info()
#deleteing missing Age details, As practically survival rate might get influenced by Age of the passanger, So decided not to impute

In [ ]:
df.dropna(subset=['Age'],inplace=True)

In [ ]:
df.info()
#PassengerID, Name, Ticket do not matter for suvival/dead, As there are just Values and catogorical in this dataset, so planned to drop given for a paggender

In [ ]:
df.drop(['Name'],inplace=True,axis=1)
df.drop(['PassengerId'],inplace=True,axis=1)
df.drop(['Ticket'],inplace=True,axis=1)
df.drop(['Cabin'],inplace=True,axis=1)

In [ ]:
df.shape

In [ ]:
df['Survived'].value_counts()
#Feel it is balanced

In [ ]:
df.describe()

In [ ]:
Corr=df.corr()
plt.figure(figsize=(6,5))
sns.heatmap(Corr,annot=True)
plt.show()
#This Map shows that the columns are very less correlation betwee the attributes

In [ ]:
plt.figure(figsize=(20,20),facecolor='white')
plotnumber=1

for column in df:
    if plotnumber<=12:
        ax=plt.subplot(3,4,plotnumber)
        sns.distplot(df[column])
        plt.xlabel(column,fontsize=20)
    plotnumber+=1
plt.tight_layout()  

#Catogorical columns-Noting to do with distibution-pClass, Sex, Sibsi, Parch, Embarked
#would check for skewness for Age and fare


In [ ]:
x_normalized=pd.DataFrame({})
x_normalized['Age']=df['Age']
x_normalized['Fare']=df['Fare']
x_normalized.skew().sort_values(ascending=False)
#There need a screwness correction for Fare

In [ ]:
x_normalized.drop(['Age'],inplace=True,axis=1)


In [ ]:
x_normalized.info()

In [ ]:
#Screwness Correction
from sklearn.preprocessing import power_transform
x_new=power_transform(x_normalized)
#x_new=pd.DataFrame(x_new,columns=['Fare'])
df['Fare']=x_new


In [ ]:
df.info()

In [ ]:
x=df.drop(columns=['Survived'],axis=1)
y=df['Survived']
x

In [ ]:
scaler=StandardScaler()
x_scaled=scaler.fit_transform(x)
x_scaled.shape

In [ ]:
x_scaled.shape

#Would Apply PCA technique as there is very less correlation of features with the target, Soo Principal component Analysis would reduce the dimentions of the features.
because I really do not understand which features would help me to predict, the features look very unrealted

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca=PCA()
components=pca.fit_transform(x_scaled)

In [ ]:
plt.figure
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('princial_components')
plt.ylabel('variance covered')
plt.title('PCA')
plt.show()
#I will use all 6 componets for model building 

In [ ]:
principl_component=pd.DataFrame(componets,columns=['PC1','PC2','PC3','PC4','PC5','PC6','PC7'])
principl_component.drop(['PC7'],inplace=True,axis=1)
principl_component

In [ ]:
#Train the data to the model
x_train, x_test, y_train, y_test=train_test_split(principl_component,y,test_size=0.25, random_state=100)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF=RandomForestClassifier()
RF.fit(x_train,y_train)
y_pred=RF.predict(x_test)
print(classification_report(y_test,y_pred,digits=2))

In [ ]:
from sklearn.ensemble import BaggingClassifier
bclf=BaggingClassifier()
bclf.fit(x_train,y_train)
y_pred=bclf.predict(x_test)
print(classification_report(y_test,y_pred,digits=2))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier()
knn.fit(x_train,y_train)
y_pred=knn.predict(x_test)

print(classification_report(y_test,y_pred,digits=2))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier()
clf.fit(x_train,y_train)
y_pred=clf.predict(x_test)
print(classification_report(y_test,y_pred,digits=2))

In [ ]:
from sklearn.svm import SVC
svc=SVC()
svc.fit(x_train,y_train)
y_pred=svc.predict(x_test)
print(classification_report(y_test,y_pred,digits=2))

In [ ]:
#Will Go with SVC as its presion on both the labels is equally good.
#Hypertunning SVC

In [ ]:
params={'C':[ 1, 3, 5, 7, 8,  12, 15, 18, 20],
   'gamma':[0.1, 0.4, 0.8, 1]}
GSCV=GridSearchCV(svc,params)
GSCV.fit(x_train,y_train)
GSCV.best_params_

In [ ]:
from sklearn.svm import SVC
svc=SVC(C=1, gamma=0.1)
svc.fit(x_train,y_train)
y_pred=svc.predict(x_test)
print(classification_report(y_test,y_pred,digits=2))

In [ ]:
#svc.get_params().keys()

In [ ]:
with open("svc", "wb") as f:
    pickle.dump(svc, f)